# Feature Selection and LASSO (Interpretation)

In this notebook, lets use LASSO to select features, building on a pre-implemented solver for LASSO You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the next notebook, we will implement your own LASSO solver, using coordinate descent. 

In [22]:
import pandas as pd
import numpy as np
from math import log, sqrt
from sklearn import linear_model
from sklearn import cross_validation

In [3]:
sales = pd.read_csv('data/kc_housing_sales_data.csv')
sales.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639


In [21]:
def get_numpy_data(data_frame, features, output):
    
    data_frame['constant'] = 1 # this is how you add a constant column to a data frame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
   
    # select the columns of data_frame given by the features list into the data frame features_frame (now including constant):
    features_frame = data_frame[features]
       
    # the following line will convert the features_frame into a numpy matrix:
    #feature_matrix = features_frame.to_numpy()
    feature_matrix = features_frame.as_matrix()
       
    # assign the column of data_frame associated with the output to the array output_array
    output_array = data_frame[output]
    
    # the following will convert the array into a numpy array by first converting it to a list
    output_array = output_array.as_matrix()
    
    return(feature_matrix, output_array)

## Create new features

Lets consider features that are some transformations of inputs.

In [6]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

## Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [7]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [18]:
model = linear_model.Lasso(alpha=1e8)
X = sales[all_features]
y = sales['price']
model.fit(X, y)

print model.coef_
print model.intercept_

[  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.61142560e+02
   0.00000000e+00   1.18887531e-01   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -0.00000000e+00
   0.00000000e+00]
203131.744589


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection.

## Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

In [20]:
training_and_validation, testing = cross_validation.train_test_split(sales, test_size=0.1, train_size=0.9, random_state=1) # initial train/test split
training, validation = cross_validation.train_test_split(training_and_validation, test_size=0.5, train_size=0.5, random_state=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [30]:
for l1_penalty in np.logspace(1, 7, num=13):
    model1 = linear_model.Lasso(alpha=l1_penalty)
    X = training[all_features]
    y = training['price']
    model1.fit(X, y)
     
    Xv = validation[all_features]
    predictions = model1.predict(Xv)
    errors = predictions - validation['price']
    errors_squared = errors * errors
    rss = sum(errors_squared)
    
    print str(l1_penalty) + "                 " + str(rss)

10.0                 4.16631132904e+14
31.6227766017                 4.16683655393e+14
100.0                 4.1687180435e+14
316.227766017                 4.17687366227e+14
1000.0                 4.20811391961e+14
3162.27766017                 4.36633951781e+14
10000.0                 4.47968780082e+14
31622.7766017                 4.84769265111e+14
100000.0                 6.00215411156e+14
316227.766017                 6.06025586962e+14
1000000.0                 6.27160334454e+14
3162277.66017                 6.48591957221e+14
10000000.0                 6.55479583628e+14


Notice best l1_penalty = 10
Get the coefficents and intercepts of the corresponding model.

In [31]:
best_model = linear_model.Lasso(alpha=10)
X = training[all_features]
y = training['price']
best_model.fit(X, y)

print best_model.coef_
print best_model.intercept_

[ -1.53172468e+04   5.21519307e+02   5.60830419e+04   2.89790484e+02
  -4.69011296e+04   6.40966751e-01  -6.55752298e+02  -9.88875585e+04
   3.49261145e+04   4.55537885e+05   3.69020245e+04   2.85242927e+04
   1.29078141e+05   3.44275684e+02   3.46088948e+02  -3.21187479e+03
   1.57649945e+01]
6583595.36258


## Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, we are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [33]:
max_nonzeros = 7

### Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [35]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. 
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [66]:
l1_penalties = []
for l1_penalty in np.logspace(8, 10, num=20):
    
    X = training[all_features]
    y = training['price']
    
    model3 = linear_model.Lasso(alpha=l1_penalty)
    model3.fit(X, y)
    
    non_zero_prameters = 0
    for cof in model3.coef_:
        if cof == 0.0:
            non_zero_prameters = non_zero_prameters + 1

    if non_zero_prameters > max_nonzeros:
        l1_penalties.append(l1_penalty)
        #print non_zero_prameters, l1_penalty

l1_penalty_max = max(l1_penalties)
l1_penalty_min = min(l1_penalties)

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [68]:
print "l1_penalty_min: " + str(l1_penalty_min)
print "l1_penalty_max: " + str(l1_penalty_max)

l1_penalty_min: 100000000.0
l1_penalty_max: 10000000000.0


### Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [72]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [92]:
for l1_penalty in l1_penalty_values:
    #print l1_penalty
    
    X = training[all_features]
    y = training['price']
    
    model3 = linear_model.Lasso(alpha=l1_penalty)
    model3.fit(X, y)
    
    non_zero_prameters = 0
    
    for cof in model3.coef_:
        if cof == 0.0:
            non_zero_prameters = non_zero_prameters + 1

    #print non_zero_prameters
    if non_zero_prameters > 7:
        print l1_penalty
        print model3.coef_
    

100000000.0
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.59261529e+02
   0.00000000e+00   1.08511234e-01   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -0.00000000e+00
   0.00000000e+00]
621052631.579
[ 0.          0.          0.          0.          0.          0.42335224
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
1142105263.16
[ 0.          0.          0.          0.          0.          0.11272554
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
1663157894.74
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
2184210526.32
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
2705263157.89
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
3226315789.47